In [1]:
from ptrail.core.Datasets import Datasets
from ptrail.features.kinematic_features import KinematicFeatures

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from src.utils.general_utils import Utilities
from TestUtils.test_utils import TestUtils
from TestUtils.Keys import *

In [2]:
traffic_data = Datasets.load_traffic_data()
ready_dataset = KinematicFeatures.create_distance_column(traffic_data)

------------------------ Dataset Facts ------------------------------

Number of unique Trajectories in the data: 125
Number of points in the data: 44905
Dataset time range: 0 days 00:00:59.900000
Datatype of the DataFrame: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>
Dataset Bounding Box: (34.7107417, 135.4640652, 34.7156517, 135.4702002)

---------------------------------------------------------------------


In [3]:
from ptrail.core.Datasets import Datasets
from ptrail.features.kinematic_features import KinematicFeatures
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from src.utils.general_utils import Utilities
from TestUtils.test_utils import TestUtils
from TestUtils.Keys import *

# Get the 20 seed values that we are going to use.
seed_generator = Utilities.generate_pi_seed(20)
seed_vals = [next(seed_generator) for i in range(20)]
final_results = ["seed, strategy, model, accuracy, f1_score"]

# All our selection strategies.
select_strategies = [
    BASE, RANDOM_SELECTED, PROPORTIONAL_SELECTED, FEWEST_SELECTED, REPRESENTATIVE_SELECTED,
    BALANCED_ON, BALANCED_IN
]
augment_strategies = [ON, IN, DROP, STRETCH]
models = [GradientBoostingClassifier(), DecisionTreeClassifier(), SVC()]

for seed in seed_vals:
    # Get the iterable map for the seed.
    iter_map = TestUtils.get_iterable_map(ready_dataset, seed, 'vehicle_type')
    for select_strategy in select_strategies:
        for model in models:
            if select_strategy != BASE and 'balanced' not in select_strategy:
                for augment_strategy in augment_strategies:
                    train_x, train_y = TestUtils.select_correct_test_train_split(iter_map, select_strategy,
                                                                                 augment_strategy,
                                                                                 'vehicle_type', 3)
                    if (train_x is not None) and (train_y is not None):
                        # Fit the model and predict.
                        model.random_state = seed
                        model.fit(X=train_x, y=train_y)
                        pred_vals = model.predict(X=iter_map[TEST_X])

                        # Calculate the accuracy and f1 score.
                        acc = accuracy_score(y_true=iter_map[TEST_Y], y_pred=pred_vals)
                        score = f1_score(y_true=iter_map[TEST_Y], y_pred=pred_vals, average='weighted')
                        print(f"{seed}, {select_strategy}_{augment_strategy}, {model.__class__.__name__}, {acc}, {score}")
                        final_results.append(f"{seed}, {select_strategy}_{augment_strategy}, {model.__class__.__name__}, {acc}, {score}")
            else:
                train_x, train_y = TestUtils.select_correct_test_train_split(iter_map, select_strategy,
                                                                             BASE, 'vehicle_type', 3)
                if (train_x is not None) and (train_y is not None):
                    # Fit the model and predict.
                    model.random_state = seed
                    model.fit(X=train_x, y=train_y)
                    pred_vals = model.predict(X=iter_map[TEST_X])

                    # Calculate the accuracy and f1 score.
                    acc = accuracy_score(y_true=iter_map[TEST_Y], y_pred=pred_vals)
                    score = f1_score(y_true=iter_map[TEST_Y], y_pred=pred_vals, average='weighted')
                    if 'balanced' not in select_strategy:
                        print(f"{seed}, base, {model.__class__.__name__}, {acc}, {score}")
                        final_results.append(f"{seed}, base,"f" {model.__class__.__name__}, {acc}, {score}")
                    else:
                        print(f"{seed}, {select_strategy}, {model.__class__.__name__}, {acc}, {score}")
                        final_results.append(f"{seed}, {select_strategy},"f" {model.__class__.__name__}, {acc}, {score}")


1415, base, GradientBoostingClassifier, 0.875, 0.8944444444444444
1415, base, DecisionTreeClassifier, 0.875, 0.8944444444444444
1415, base, SVC, 0.9583333333333334, 0.9379432624113475
1415, random_selected_on, GradientBoostingClassifier, 0.8333333333333334, 0.8712121212121211
1415, random_selected_in, GradientBoostingClassifier, 0.7916666666666666, 0.8468992248062017
1415, random_selected_drop, GradientBoostingClassifier, 0.9583333333333334, 0.9379432624113475
1415, random_selected_stretch, GradientBoostingClassifier, 0.8333333333333334, 0.8712121212121211
1415, random_selected_on, DecisionTreeClassifier, 0.875, 0.8944444444444444
1415, random_selected_in, DecisionTreeClassifier, 0.7916666666666666, 0.8468992248062017
1415, random_selected_drop, DecisionTreeClassifier, 0.9166666666666666, 0.9166666666666666
1415, random_selected_stretch, DecisionTreeClassifier, 0.7916666666666666, 0.8533681765389084
1415, random_selected_on, SVC, 0.9583333333333334, 0.9379432624113475
1415, random_sele

In [4]:
TestUtils.write_csv_file('./traffic_results.csv', final_results)

File successfully written to: ./traffic_results.csv
